In [2]:
import pandas as pd
df = pd.read_csv('data.csv').drop(['jerseyNumber'], axis=1)
'''
Columns (distance not considered):
playId
initiator(role==1)
receiver(role==2)
3 closest opponent to the initiator
3 closest opponent to the receiver
result
'''
df.head(22)

,x,y,speed,distance,direction,role,playId,result
0,31.46,23.50,0.86,0.13,277.47,3,51,3
1,34.89,34.87,1.26,0.14,130.14,3,51,3
2,37.95,34.02,0.16,0.09,51.23,3,51,3
3,40.47,45.10,2.45,0.37,65.64,3,51,3
4,32.80,26.70,3.01,0.31,223.39,3,51,3
5,32.27,26.60,2.44,0.23,250.53,3,51,3
6,35.72,29.89,1.41,0.22,259.69,3,51,3
7,41.78,13.06,3.71,0.38,183.47,3,51,3
8,34.06,29.86,2.24,0.29,212.67,3,51,3
9,54.87,27.38,4.04,0.43,172.55,3,51,3


In [2]:
# %pip show numpy
# %pip uninstall numpy
%pip install numpy==1.22.0

   ---------------------------------------- 0.0/14.7 MB ? eta -:--:--
   ----- ---------------------------------- 2.1/14.7 MB 23.5 MB/s eta 0:00:01
   -------------- ------------------------- 5.2/14.7 MB 13.9 MB/s eta 0:00:01
   ------------------- -------------------- 7.1/14.7 MB 11.8 MB/s eta 0:00:01
   ------------------------- -------------- 9.4/14.7 MB 14.3 MB/s eta 0:00:01
   ---------------------------------------- 14.7/14.7 MB 15.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.0
    Uninstalling numpy-1.23.0:
      Successfully uninstalled numpy-1.23.0
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
numba 0.60.0 requires llvmlite<0.44,>=0.43.0dev0, but you have llvmlite 0.38.0 which is incompatible.
rasterio 1.4.1 requires numpy>=1.24, but you have numpy 1.22.0 which is incompatible.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.22.0 which is incompatible.
ultralytics 8.0.235 requires numpy>=1.22.2, but you have numpy 1.22.0 which is incompatible.


In [4]:
import numpy as np

def dist_between_points(p1:tuple, p2:tuple):
    return np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

In [15]:
pd.options.mode.chained_assignment = None

# _d is direction, not distance
new_df = pd.DataFrame(columns=[
    'playId', 
    'init_x', 'init_y', 'init_s', 'init_d', 
    'recv_x', 'recv_y', 'recv_s', 'recv_d', 
    'init_opp_1_x', 'init_opp_1_y', 'init_opp_1_s', 'init_opp_1_d', 
    'init_opp_2_x', 'init_opp_2_y', 'init_opp_2_s', 'init_opp_2_d', 
    'init_opp_3_x', 'init_opp_3_y', 'init_opp_3_s', 'init_opp_3_d', 
    'recv_opp_1_x', 'recv_opp_1_y', 'recv_opp_1_s', 'recv_opp_1_d', 
    'recv_opp_2_x', 'recv_opp_2_y', 'recv_opp_2_s', 'recv_opp_2_d', 
    'recv_opp_3_x', 'recv_opp_3_y', 'recv_opp_3_s', 'recv_opp_3_d', 
    'result'])

block_size = 22
for i in range(0, len(df), block_size):
    block = df.iloc[i:i + block_size]

    init = block[block['role'] == 1]
    recv = block[block['role'] == 2]

    if init.empty or recv.empty:
        raise Exception(f"init or recv empty in {i//22}th block")
    
    if len(init) != 1 or len(recv) != 1:
        raise Exception(f"init or recv more than 1 in {i//22}th block")
    
    new_row = []
    new_row.append(block.head(1)['playId'].item())

    home_count = 0
    for index, row in block.iterrows():
        if row['role'] == 0:
            home_count+=1
    
    if home_count == 9:
        opponent = 3
    elif home_count == 11:
        opponent = 0
    else:
        raise Exception(f"Home count is {home_count} in the {i//22}th block")
    
        
    init_x = init.iloc[0]['x']
    init_y = init.iloc[0]['y']
    recv_x = recv.iloc[0]['x']
    recv_y = recv.iloc[0]['y']

    new_row.extend([init_x, init_y, init.iloc[0]['speed'], init.iloc[0]['direction']])
    new_row.extend([recv_x, recv_y, recv.iloc[0]['speed'], recv.iloc[0]['direction']])

    block.loc[:, 'dist_to_init'] = 0.0
    block.loc[:, 'dist_to_recv'] = 0.0

    for index, row in block.iterrows():
        current_x = row['x']
        current_y = row['y']

        dist_to_init = dist_between_points((current_x, current_y), (init_x, init_y))
        dist_to_recv = dist_between_points((current_x, current_y), (recv_x, recv_y))
        block.loc[index, 'dist_to_init'] = dist_to_init
        block.loc[index, 'dist_to_recv'] = dist_to_recv
    
    opponents = block[block['role'] == opponent]

    closest_init = opponents.sort_values('dist_to_init').iloc[1:4]
    closest_recv = opponents.sort_values('dist_to_recv').iloc[1:4]
    
    for index, row in closest_init.iterrows():
        new_row.extend([row['x'], row['y'], row['speed'], row['direction']])

    for index, row in closest_recv.iterrows():
        new_row.extend([row['x'], row['y'], row['speed'], row['direction']])

    new_row.append(block.head(1)['result'].item())

    new_df.loc[len(new_df)] = new_row

new_df

,playId,init_x,init_y,init_s,init_d,recv_x,recv_y,recv_s,recv_d,init_opp_1_x,...,recv_opp_1_d,recv_opp_2_x,recv_opp_2_y,recv_opp_2_s,recv_opp_2_d,recv_opp_3_x,recv_opp_3_y,recv_opp_3_s,recv_opp_3_d,result
0,51.0,30.22,29.28,1.23,254.73,38.97,14.15,5.93,149.11,32.80,...,163.44,31.46,23.50,0.86,277.47,32.80,26.70,3.01,223.39,3.0
1,229.0,43.41,28.69,1.15,117.37,53.83,16.61,3.32,145.75,40.10,...,167.40,63.80,11.24,3.86,81.35,65.11,19.90,2.63,206.29,1.0
2,287.0,55.64,24.70,0.42,186.92,63.52,35.01,6.55,20.20,57.19,...,22.70,61.95,25.93,0.60,275.73,56.05,29.20,2.07,246.88,1.0
3,336.0,60.88,28.90,3.34,248.92,69.68,24.25,6.56,160.29,64.31,...,40.93,65.28,24.31,2.11,272.13,74.07,23.00,2.82,350.23,1.0
4,427.0,92.37,23.89,1.62,248.26,96.35,32.01,4.20,143.00,94.84,...,192.56,95.27,24.51,2.50,183.74,97.60,23.49,0.58,200.38,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,4330.0,104.44,3.42,6.35,154.59,116.44,9.73,5.81,168.01,107.93,...,185.79,114.08,1.51,2.42,135.25,107.93,5.59,7.09,187.08,2.0
59,4432.0,100.11,24.72,1.05,316.55,110.59,38.84,0.60,8.76,104.19,...,126.14,101.61,28.64,2.11,249.59,110.86,24.79,4.17,163.12,3.0
60,4476.0,50.65,24.20,3.52,276.26,60.23,37.05,4.21,165.06,51.75,...,201.08,64.60,27.24,3.14,120.81,51.75,28.37,2.29,266.89,2.0
61,4502.0,50.98,22.95,2.62,260.13,54.86,15.85,0.87,167.52,53.83,...,237.70,53.83,22.99,4.82,232.98,52.90,24.02,3.63,227.75,1.0


In [14]:
new_df.to_csv('mlp_data.csv', index=False)

In [3]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
import os


In [ ]:

new_df = pd.read_csv('mlp_data_rounded.csv')
num_classes = 6
model_folder = 'models/'
data_folder = 'datas/'

x_train, x_test, y_train, y_test = train_test_split(new_df.iloc[:, :-1], new_df.iloc[:, -1], test_size=0.2, random_state=0)
joblib.dump(x_test, os.path.join(data_folder, 'x_test.joblib'))
joblib.dump(y_test, os.path.join(data_folder, 'y_test.joblib'))

# Hyperparameters
LAYERS = [
    [[64, 'relu'], [32, 'relu']],
    [[128, 'relu'], [64, 'relu'], [32, 'relu']],
    [[64, 'relu'], [64, 'relu'], [32, 'relu']],
    [[256, 'relu'], [128, 'relu'], [64, 'relu'], [32, 'relu']],
    [[128, 'tanh'], [64, 'tanh']],
    [[64, 'sigmoid'], [32, 'sigmoid']],
    [[128, 'relu'], [64, 'relu'], [32, 'relu'], [16, 'relu']],
    [[64, 'relu'], [64, 'tanh'], [32, 'relu']],
    [[128, 'relu'], [64, 'relu'], [32, 'sigmoid']],
    [[64, 'relu'], [32, 'relu'], [16, 'relu'], [8, 'relu']],
    [[128, 'relu'], [64, 'relu'], [32, 'relu'], [32, 'relu']],
]


EPOCHS = [20, 30, 50, 100]
BATCH_SIZES = [10, 20, 30]

count = 0
for layers in LAYERS:
    count+=1
    for epoch in EPOCHS:
        for batch_size in BATCH_SIZES:
            print(f'Creating model with layers: {layers}, epoch: {epoch}, batch size: {batch_size}...')
            model = tf.keras.Sequential()
            model.add(tf.keras.layers.Input(shape=(33,)))
            for i in range(1, len(layers)-1):
                model.add(tf.keras.layers.Dense(layers[i][0], activation=layers[i][1]))
            model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
            print('Model created')
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            print('Start training...')
            model.fit(x_train, y_train, epochs=epoch, batch_size=batch_size)
            print('Training complete, saving model...')
            path = os.path.join(model_folder, f'mlp_layer_{count}_{epoch}_{batch_size}.joblib')
            joblib.dump(model, path)
            print('Model saved')
print('All iterations completed')

Creating model with layers: [[64, 'relu'], [32, 'relu']], epoch: 20, batch size: 10...
Model created
Start training...
Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0231 - loss: 1237.4932      
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0272 - loss: 1159.7800     
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0314 - loss: 926.1276     
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0356 - loss: 1071.7825     
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0494 - loss: 1038.8451     
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0328 - loss: 902.4392     
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0494 - loss: 864.9705     
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0381 - loss: 881.3665     
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0561 - loss: 873.2599     
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0658 - loss:

In [1]:
import os
import tensorflow as tf
import numpy as np
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


In [8]:

highest_accuracy = 0
highest_accuracy_model = ''
highest_precision = 0
highest_precision_model = ''
highest_recall = 0
highest_recall_model = ''
highest_f1 = 0
highest_f1_model = ''

for model_fname in os.listdir(model_folder):
    no_ext = model_fname.split('.')
    fname = no_ext[0].split('_')
    f_layers = LAYERS[int(fname[2])-1]
    f_epoch = int(fname[3])
    f_batch_size = int(fname[4])

    model_path = os.path.join(model_folder, model_fname)
    model = joblib.load(model_path)
    y_pred = np.argmax(model.predict(x_test), axis=1)
    accuracy = accuracy_score(y_test, y_pred)
    if accuracy > highest_accuracy:
        highest_accuracy = accuracy
        highest_accuracy_model = model_fname
    precision = precision_score(y_test, y_pred, average='weighted')
    if precision > highest_precision:
        highest_precision = precision
        highest_precision_model = model_fname
    recall = recall_score(y_test, y_pred, average='weighted')
    if recall > highest_recall:
        highest_recall = recall
        highest_recall_model = model_fname
    f1 = f1_score(y_test, y_pred, average='weighted')
    if f1 > highest_f1:
        highest_f1 = f1
        highest_f1_model = model_fname

    with open(os.path.join(data_folder, 'results.txt'), 'a') as file:
        file.write(model_fname)
        file.write('\n')
        file.write('Parameters:\n')
        file.write(f'\tLayers: \n')
        for layer in f_layers:
            file.write(f'\t\tUnits: {layer[0]}\tActivation: {layer[1]}\n')
        file.write(f'\tEpochs: {f_epoch}\n')
        file.write(f'\tBatch size: {f_batch_size}\n')
        file.write('\n')
        file.write(f'Accuracy: {round(accuracy*100, 2)}\n')
        file.write(f'Average precision: {round(precision*100, 2)}\n')
        file.write(f'Average recall: {round(recall*100, 2)}\n')
        file.write(f'Average F1-score: {round(f1*100, 2)}\n')
        file.write('\n')
        file.write('===============================================================================================================================\n')
        file.write('\n')
print('Logging completed')
print(f'{highest_accuracy_model} with accuracy: {round(highest_accuracy*100, 2)}')
print(f'{highest_precision_model} with precision: {round(highest_precision*100, 2)}')
print(f'{highest_recall_model} with recall: {round(highest_recall*100, 2)}')
print(f'{highest_f1_model} with f1: {round(highest_f1*100, 2)}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


c:\Users\ivslab\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ivslab\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


c:\Users\ivslab\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ivslab\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ivslab\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


...

c:\Users\ivslab\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ivslab\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ivslab\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


c:\Users\ivslab\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ivslab\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Logging completed
mlp_layer_5_100_30.joblib with accuracy: 61.54
mlp_layer_6_30_30.joblib with precision: 79.49
mlp_layer_5_100_30.joblib with recall: 61.54
mlp_layer_5_100_30.joblib with f1: 67.06


c:\Users\ivslab\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ivslab\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
# Models with rounded data performed worse for some reason, only at 60%
df = pd.read_csv('mlp_data.csv')
df['init_x'] = df['init_x'].round()
df['init_y'] = df['init_y'].round()
df['recv_x'] = df['recv_x'].round()
df['recv_y'] = df['recv_y'].round()
df['init_opp_1_x'] = df['init_opp_1_x'].round()
df['init_opp_1_y'] = df['init_opp_1_y'].round()
df['init_opp_2_x'] = df['init_opp_2_x'].round()
df['init_opp_2_y'] = df['init_opp_2_y'].round()
df['init_opp_3_x'] = df['init_opp_3_x'].round()
df['init_opp_3_y'] = df['init_opp_3_y'].round()
df['recv_opp_1_x'] = df['recv_opp_1_x'].round()
df['recv_opp_1_y'] = df['recv_opp_1_y'].round()
df['recv_opp_2_x'] = df['recv_opp_2_x'].round()
df['recv_opp_2_y'] = df['recv_opp_2_y'].round()
df['recv_opp_3_x'] = df['recv_opp_3_x'].round()
df['recv_opp_3_y'] = df['recv_opp_3_y'].round()

df.to_csv('mlp_data_rounded.csv', index=False)
